In [20]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from datetime import datetime
import numpy as np
import pandas as pd

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)
my_index = "network_weather-2017.9.*"

my_query = {}

# CHANGE SO IT RETREIVES ONLY UNIQUE hashes.

# sS='UC'
# srcSiteOWDServer = "192.170.227.160"
# srcSiteThroughputServer = "192.170.227.162"

sS='CERN-PROD'
srcSiteOWDServer = "128.142.223.247"
srcSiteThroughputServer = "128.142.223.246"

# dS='IU'
# destSiteOWDServer = "149.165.225.223"
# destSiteThroughputServer = "149.165.225.224"

# dS='UIUC'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.96.9"

# dS='ICCN'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.126.132"

dS='pic'
destSiteOWDServer = "193.109.172.188"
destSiteThroughputServer = "193.109.172.187"

#grab user system current date
current_date = datetime.today()

#correct 1 digit days/months to DD/MM format
user_yr = str(current_date.year)
user_month = str(current_date.month)
if len(user_month) < 2:
    user_month = '0' + user_month
user_day = str(current_date.day)
if len(user_day) < 2:
    user_day = '0' + user_day
end_date = user_yr + user_month + user_day + 'T000000Z'

#subtract 1 week from current date
begin_raw = current_date - timedelta(weeks=10)

#do same corrections as above
start_yr = str(begin_raw.year)
start_month = str(begin_raw.month)
if len(start_month) < 2:
    start_month = '0' + start_month
start_day = str(begin_raw.day)
if len(start_day) < 2:
    start_day = '0' + start_day
start_date = start_yr + start_month + start_day + 'T000000Z'

my_query = {
    "size":1,
    "_source": {
        "include": [ 'src','dest','hops','hash' ]
    },
    'query':{
        'bool':{
            'must':[
                {'range': {'timestamp': {'gte': start_date, 'lt': end_date}}},
                {'term': {'_type': 'traceroute'}},
#                         {'bool':
#                             {'should':[
#                                 {'term': {'src': srcSiteOWDServer}},
#                                 {'term': {'src': srcSiteThroughputServer}},
#                                 {'term': {'src': destSiteOWDServer}},
#                                 {'term': {'src': destSiteThroughputServer}}
#                             ]}
#                         }
#                         ,
#                         {'bool':
#                             {'should':[
#                                 {'term': {'dest': destSiteOWDServer}},
#                                 {'term': {'dest': destSiteThroughputServer}},
#                                 {'term': {'dest': srcSiteOWDServer}},
#                                 {'term': {'dest': srcSiteThroughputServer}}
#                             ]}
#                         }
            ]

        }
    },
    "aggs": {
        "grouped_by_hash": {
          "terms": {  "field": "hash", "size":10000 }, #
          "aggs": {
              "top_hash_hits": {
                  "top_hits": {
                      "sort": [ { "_score": { "order": "desc" } } ],
                      "size": 1
                  }
              }
          }
       }
    }
}

results = es.search(body=my_query, index=my_index, request_timeout=12000)
print(len(results['aggregations']['grouped_by_hash']['buckets']))

10000


In [73]:
total_data = len(results['aggregations']['grouped_by_hash']['buckets'])

hop_grab = []
num_hops = []
rtt_grab = []
ttl_grab = []

for i in range(0,total_data):
    hop_grab.append(results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['hops'])
    num_hops.append(results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['n_hops'])
    rtt_grab.append(results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['rtts'])
    ttl_grab.append(results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['ttls'])
    
print('ran')    

ran


In [90]:
#TEST CELL FOR INDIVIDUAL HOPS
hop_dict = {}

inner_hop_dict = {}

x = 7144
start_loc = results['aggregations']['grouped_by_hash']['buckets'][x]['top_hash_hits']['hits']['hits'][0]['_source']['src_host']
end_loc = results['aggregations']['grouped_by_hash']['buckets'][x]['top_hash_hits']['hits']['hits'][0]['_source']['destSite']
print(end_loc)
location_key = start_loc + ' to ' + end_loc
for j in range(0, num_hops[x]-1):
    inner_hop_dict[j] = {location_key: hop_grab[x][j]}

hop_dict[0] = inner_hop_dict
print(hop_dict)

INFN-T1
{0: {0: {'hcc-ps02.unl.edu to INFN-T1': '129.93.239.190'}, 1: {'hcc-ps02.unl.edu to INFN-T1': '198.124.80.33'}, 2: {'hcc-ps02.unl.edu to INFN-T1': '62.40.126.153'}, 3: {'hcc-ps02.unl.edu to INFN-T1': '62.40.126.152'}, 4: {'hcc-ps02.unl.edu to INFN-T1': '62.40.126.202'}, 5: {'hcc-ps02.unl.edu to INFN-T1': '193.206.128.81'}, 6: {'hcc-ps02.unl.edu to INFN-T1': '90.147.80.106'}}}


In [93]:
hop_dict = {}

for i in range(0,total_data):
    inner_hop_dict = {}
    start_loc = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['src_host']
    #NOTE: not every data point seems to have "destSite" as a filled parameter, otherwise that might be preferable
    end_loc = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['dest']
    location_key = start_loc + ' to ' + end_loc
    for j in range(0, num_hops[i]):
        #inner_hop_dict[j] = {location_key: hop_grab[i][j]}
        #above seems to be ideal but exceeds data rate
        
        #alternative:
        inner_hop_dict[j] = hop_grab[i][j]
        #but this seems redundant?
    hop_dict[i] = inner_hop_dict

print(hop_dict)

{0: {0: '72.36.96.1', 1: '130.126.1.109', 2: '72.36.80.30', 3: '146.57.252.194', 4: '146.57.252.222', 5: '146.57.252.225', 6: '146.57.253.217', 7: '129.93.5.165'}, 1: {0: '192.41.230.1', 1: '207.72.112.89', 2: '146.57.253.30', 3: '146.57.252.213', 4: '146.57.252.222', 5: '146.57.252.225', 6: '146.57.253.217', 7: '129.93.5.165'}, 2: {0: '128.40.4.245', 1: '128.40.20.149'}, 3: {0: '18.12.0.1', 1: '18.192.41.1'}, 4: {0: '128.40.4.245', 1: '128.40.20.21'}, 5: {0: '149.165.225.254', 1: '149.165.254.197', 2: '149.165.255.8', 3: '198.124.80.97', 4: '198.124.80.85', 5: '198.124.80.86', 6: '130.126.1.110', 7: '72.36.96.9'}, 6: {0: '72.36.96.15'}, 7: {0: '192.170.226.1', 1: '192.170.224.122', 2: '198.32.11.112', 3: '164.113.255.249', 4: '164.113.255.46', 5: '129.93.5.165'}, 8: {0: '72.36.96.1', 1: '130.126.1.109', 2: '198.124.80.85', 3: '198.124.80.49', 4: '198.124.80.50', 5: '10.224.2.46', 6: '192.12.15.111'}, 9: {0: '149.165.225.254', 1: '149.165.254.249', 2: '149.165.227.22', 3: '192.170.224.

In [ ]:
buckets=results['aggregations']['grouped_by_hash']['buckets']
results=[]
# c=0
for b in buckets:
#     if c==0: 
#         print(b)
#         c+=1
    if b['doc_count']>20: # we don't want very short jumps
        results.append(b['top_hash_hits']['hits']['hits'][0])